# Assignment 4
## Understaning scaling of linear algebra operations on Apache Spark using Apache SystemML

In this assignment we want you to understand how to scale linear algebra operations from a single machine to multiple machines, memory and CPU cores using Apache SystemML. Therefore we want you to understand how to migrate from a numpy program to a SystemML DML program. Don't worry. We will give you a lot of hints. Finally, you won't need this knowledge anyways if you are sticking to Keras only, but once you go beyond that point you'll be happy to see what's going on behind the scenes.

So the first thing we need to ensure is that we are on the latest version of SystemML, which is 1.2.0:

The steps are:
- pip install
- start execution at the cell with the version - check

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')
    


In [2]:
!pip install pyspark==3.3.0

In [3]:
!pip install systemds==2.2.1

In [4]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType, IntegerType, StringType
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/02/11 08:19:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import numpy as np
u = np.random.rand(1000,10000)
s = np.random.rand(10000,1000)
w = np.random.rand(1000,1000)

Now we implement a short one-liner to define a very simple linear algebra operation

In case you are unfamiliar with matrxi-matrix multiplication: https://en.wikipedia.org/wiki/Matrix_multiplication

sum(U' * (W . (U * S)))


| Legend        |            |   
| ------------- |-------------| 
| '      | transpose of a matrix | 
| * | matrix-matrix multiplication      |  
| . | scalar multiplication      |   



In [6]:
import time
start = time.time()
res = np.sum(u.T.dot(w * u.dot(s)))
print (time.time()-start)

7.7439093589782715


As you can see this executes perfectly fine. Note that this is even a very efficient execution because numpy uses a C/C++ backend which is known for it's performance. But what happens if U, S or W get such big that the available main memory cannot cope with it? Let's give it a try:

In [ ]:
#u = np.random.rand(10000,100000)
#s = np.random.rand(100000,10000)
#w = np.random.rand(10000,10000)

After a short while you should see a memory error. This is because the operating system process was not able to allocate enough memory for storing the numpy array on the heap. Now it's time to re-implement the very same operations in SystemDS, and this is your task. Just replace all ###your_code_goes_here sections with proper code, please consider the following table which contains all syntax you need:

| Syntax        |            |   
| ------------- |-------------| 
| M.t()      | transpose of a matrix, where M is the matrix | 
| `M.__matmul__(N)` | matrix-matrix multiplication between M and N      |  
| M * N | scalar multiplication between matrix M and N     |   


## Task

We use SystemDSContext to interface with Apache SystemDS (formerly SystemML). Note that we passed a SparkSession object as parameter to SystemDSContext so now it knows how how to talk to the Apache Spark cluster

In [8]:
from systemds.context import SystemDSContext


with SystemDSContext(spark) as sds:
    # Now we create some large random matrices to have numpy and SystemDS crunch on it
    U = sds.rand(rows=1000,cols=10000)
    S = sds.rand(rows=10000,cols=1000)
    W = sds.rand(rows=1000,cols=1000)
    res = (U.t().__matmul__((W * (U.__matmul__(S))))).sum()
    print(res.compute())


6250220411917.74


In order to show you the advantage of SystemML over numpy we've blown up the sizes of the matrices. Unfortunately, on a 1-2 worker Spark cluster it takes quite some time to complete. Therefore we've stripped down the example to smaller matrices below, but we've kept the code, just in case you are curious to check it out. But you might want to use some more workers which you easily can configure in the environment settings of the project within Watson Studio. Just be aware that you're currently limited to free 50 capacity unit hours per month wich are consumed by the additional workers.

To get consistent results we switch from a random matrix initialization to something deterministic

If everything runs fine you should get *6252492444241.075* as result (or something in that bullpark). Feel free to submit your solutionto the grader now!

### Submission

In [9]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2024-02-11 08:24:21--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

rklib.py            100%[===================>]   2.24K  --.-KB/s    in 0s      

2024-02-11 08:24:21 (9.19 MB/s) - ‘rklib.py’ saved [2289/2289]



In [10]:
from rklib import submit
key = "esRk7vn-Eeej-BLTuYzd0g"


email = "chiran.nalapalu@gmail.com"

In [11]:
part = "fUxc8"
token = "0oCd852ngdwoKsEP" #you can obtain it from the grader page on Coursera (have a look here if you need more information on how to obtain the token https://youtu.be/GcDo0Rwe06U?t=276)
submit(email, token, key, part, [part], res.script.dml_script.replace('\n','x'))

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"P1p3F","id":"tE4j0qhMEeecqgpT6QjMdA~P1p3F~912MJMi2Ee6mgxK6X3Mz2Q","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
